In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

my_seed = 59

/kaggle/input/cian-big-three/cian_flat_sale_1_300_moskva_13_Apr_2025_14_15_15_159105.csv
/kaggle/input/cian-big-three/cian_flat_sale_1_300_ekaterinburg_13_Apr_2025_14_15_12_486810.csv
/kaggle/input/cian-big-three/cian_flat_sale_1_300_sankt-peterburg_13_Apr_2025_14_15_09_784160.csv


In [6]:
csv_path = '/kaggle/input/cian-big-three/cian_flat_sale_1_300_ekaterinburg_13_Apr_2025_14_15_12_486810.csv'
ds_ekat = pd.read_csv(csv_path, delimiter=';')

csv_path = '/kaggle/input/cian-big-three/cian_flat_sale_1_300_sankt-peterburg_13_Apr_2025_14_15_09_784160.csv'
ds_spb = pd.read_csv(csv_path, delimiter=';')

csv_path = '/kaggle/input/cian-big-three/cian_flat_sale_1_300_moskva_13_Apr_2025_14_15_15_159105.csv'
ds_msk = pd.read_csv(csv_path, delimiter=';')

# TODO concat these .csv's

print(ds_ekat.columns)
print(len(ds_ekat))
print(len(ds_spb))
print(len(ds_msk))

Index(['author', 'author_type', 'url', 'location', 'deal_type',
       'accommodation_type', 'floor', 'floors_count', 'rooms_count',
       'total_meters', 'price', 'district', 'street', 'house_number',
       'underground', 'residential_complex'],
      dtype='object')
1402
1446
1436


In [68]:
ds

,author,author_type,url,location,deal_type,accommodation_type,floor,floors_count,rooms_count,total_meters,price,district,street,house_number,underground,residential_complex
0,Деловой Квартал,developer,https://ekb.cian.ru/sale/flat/304227421/,Екатеринбург,sale,flat,4,25,3,88.28,13478034.0,Октябрьский,NaN,NaN,Ботаническая,Премиум Кварталы Маяковка ЖК
1,Центр Ипотеки и Недвижимости Екатеринбург,real_estate_agent,https://ekb.cian.ru/sale/flat/314477752/,Екатеринбург,sale,flat,1,3,2,45.40,3400000.0,Октябрьский,Трактовая,2,Ботаническая,NaN
2,Принцип недвижимость,real_estate_agent,https://ekb.cian.ru/sale/flat/314036930/,Екатеринбург,sale,flat,4,5,1,37.90,4100000.0,Орджоникидзевский,переулок Суворовский,19,Уралмаш,NaN
3,TEN девелопмент,developer,https://ekb.cian.ru/sale/flat/313518897/,Екатеринбург,sale,flat,6,18,1,34.00,6010000.0,Верх-Исетский,NaN,NaN,Площадь 1905 года,Русь XXI ЖК
4,Центр Ипотеки и Недвижимости Екатеринбург,real_estate_agent,https://ekb.cian.ru/sale/flat/312974624/,Екатеринбург,sale,flat,10,25,2,44.80,6100000.0,NaN,Рябинина,47,Геологическая,Eleven
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,Деловой Квартал,developer,https://ekb.cian.ru/sale/flat/300548364/,Екатеринбург,sale,flat,19,25,2,55.39,9780700.0,Октябрьский,NaN,NaN,Ботаническая,Премиум Кварталы Маяковка ЖК
1398,TEN девелопмент,developer,https://ekb.cian.ru/sale/flat/313516345/,Екатеринбург,sale,flat,22,28,2,48.10,10607400.0,Центр,Свердлова,10,Динамо,Екатерининский парк
1399,TEN девелопмент,developer,https://ekb.cian.ru/sale/flat/313516571/,Екатеринбург,sale,flat,17,21,2,55.50,8295300.0,Верх-Исетский,NaN,NaN,Площадь 1905 года,Русь-2 ЖК
1400,TEN девелопмент,developer,https://ekb.cian.ru/sale/flat/314122168/,Екатеринбург,sale,flat,8,28,1,36.40,8954000.0,Центр,Свердлова,10,Динамо,Екатерининский парк


In [54]:
ds = ds.dropna(axis=0, subset=['price'])
# "location" is important if data contains different cities
redundant_columns = ['author', 'url', 'location', 'deal_type', 'accommodation_type']
ds = ds.drop(redundant_columns, axis=1)
print(len(ds))
print(ds.describe())

1390
             floor  floors_count  rooms_count  total_meters         price
count  1390.000000   1390.000000  1390.000000   1390.000000  1.390000e+03
mean      8.246043     19.495683     1.530216     49.107691  7.852342e+06
std       6.902690      8.057098     0.735910     18.937007  7.476887e+06
min       1.000000      2.000000    -1.000000     13.200000  4.950000e+05
25%       3.000000     15.000000     1.000000     38.910000  6.150000e+06
50%       6.000000     21.000000     1.000000     44.105000  6.895500e+06
75%      13.000000     25.000000     2.000000     53.480000  7.492790e+06
max      30.000000     52.000000     4.000000    241.000000  2.070000e+08


In [55]:
# drop rooms_count == -1
ds = ds[ds['rooms_count'] > 0]
print(len(ds))

1388


In [56]:
# fix NaN in cat. features
object_cols = [col for col in ds.columns if ds[col].dtype == "object"]
for col in object_cols:
    ds[col] = ds[col].fillna('undef')
    ds[col] = ds[col].str.strip().str.lower()

In [57]:
X, y = ds.drop(['price'], axis=1), ds['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=my_seed)
print(len(X_train), len(X_test), len(y_train), len(y_test))

1110 278 1110 278


In [58]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# high cardinality in some columns
sorted(d.items(), key=lambda x: x[1])

[('author_type', 7),
 ('underground', 10),
 ('district', 16),
 ('residential_complex', 181),
 ('house_number', 185),
 ('street', 186)]

In [59]:
# frequency encoding and ohe

low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 20]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print(low_cardinality_cols)
print(high_cardinality_cols)

ohe_encoding = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
for col in low_cardinality_cols:
    ohe_cats_array = ohe_encoding.fit_transform(X_train[[col]])
    ohe_cats_df = pd.DataFrame(ohe_cats_array, 
                               columns=[f"{col}_type_{cat}" for cat in ohe_encoding.categories_[0]],
                               index=X_train.index)
    X_train = pd.concat([X_train, ohe_cats_df], axis=1).drop([col], axis=1)

    ohe_test_cats_array = ohe_encoding.transform(X_test[[col]])
    ohe_test_cats_df = pd.DataFrame(ohe_test_cats_array, 
                                    columns=[f"{col}_type_{cat}" for cat in ohe_encoding.categories_[0]],
                                    index=X_test.index)
    X_test = pd.concat([X_test, ohe_test_cats_df], axis=1).drop([col], axis=1)

X_train.head()

['author_type', 'district', 'underground']
['house_number', 'residential_complex', 'street']


,floor,floors_count,rooms_count,total_meters,street,house_number,residential_complex,author_type_type_developer,author_type_type_homeowner,author_type_type_real_estate_agent,...,underground_type_undef,underground_type_ботаническая,underground_type_геологическая,underground_type_динамо,underground_type_машиностроителей,underground_type_площадь 1905 года,underground_type_проспект космонавтов,underground_type_уралмаш,underground_type_уральская,underground_type_чкаловская
1361,1,17,1,40.00,плотников,6а,белый парус,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
345,15,16,2,47.00,бебеля,166,undef,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
728,1,25,1,42.50,блюхера,89,уральский,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
916,9,31,2,44.42,undef,undef,спутник-1 жк,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1363,3,30,1,43.13,undef,undef,река. дом на набережной жк,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
# maybe normalize in .value_counts()?..
for col in high_cardinality_cols:
    freq_map = X_train[col].value_counts().to_dict()
    
    X_train[col+'_freqs'] = X_train[col].map(freq_map)
    X_train = X_train.drop(col, axis=1)

    X_test[col+'_freqs'] = X_test[col].map(freq_map)
    X_test = X_test.drop(col, axis=1)

In [65]:
X_train['residential_complex_freqs']

1361      4
345     139
728      12
916       9
1363      2
       ... 
58      139
208      13
669      16
1243      6
178     139
Name: residential_complex_freqs, Length: 1110, dtype: int64